In [ ]:
import sys
import os
sys.path.append("../")

In [ ]:
import warnings
import time
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import feature_engine.imputation as fe

In [ ]:
from lidl_x_tum_uncertainty_estimation_mqcnn.uncertainty_estimation_models import Model, MQCNN
from lidl_x_tum_uncertainty_estimation_mqcnn.constants import DistEnum, PredEnum

## 1 Data preparation

In [ ]:
import pandas as pd
df = pd.read_pickle('/dbfs/mnt/tum/data/kaggle/corporacion_favorita/favorita_full_df.pickle')

In [ ]:
df.head(1000)

,date,entity,store_nbr,item_nbr,unit_sales,onpromotion,Holiday,family,class,perishable,city,state,type,cluster,dcoilwtico,transactions,promo_missing,unit_saleslag16,unit_saleslag17,unit_saleslag18,unit_saleslag19,unit_saleslag20,unit_saleslag21,unit_saleslag22,unit_saleslag30,unit_saleslag60,mean_7,std_7,mean_30,std_30,mean_60,std_60,ramp_upHoliday,ramp_downHoliday,ramp_uponpromotion,ramp_downonpromotion
195,2016-07-15,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,45.930000,1275.0,1,2.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,9.0,1.000000,1.527525,1.566667,1.924136,1.966667,2.497230,0.0,0.0,0.370408,0.370408
196,2016-07-16,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,NaN,1219.0,1,1.0,2.0,0.0,0.0,4.0,0.0,0.0,2.0,1.0,1.000000,1.527525,1.533333,1.925032,1.950000,2.500339,0.0,0.0,0.370408,0.370408
197,2016-07-17,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,NaN,885.0,1,1.0,1.0,2.0,0.0,0.0,4.0,0.0,3.0,1.0,1.142857,1.463850,1.533333,1.925032,1.950000,2.500339,0.0,0.0,0.370408,0.370408
198,2016-07-18,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,45.230000,1445.0,1,2.0,1.0,1.0,2.0,0.0,0.0,4.0,5.0,3.0,1.428571,1.397276,1.433333,1.813424,1.866667,2.410898,0.0,0.0,0.370408,0.370408
199,2016-07-19,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,44.639999,1360.0,1,0.0,2.0,1.0,1.0,2.0,0.0,0.0,4.0,1.0,0.857143,0.899735,1.433333,1.813424,1.850000,2.420429,0.0,0.0,0.370408,0.370408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,2017-02-01,13669,15,1004550,2.0,0,0,22,1318,0,9,7,2,14,53.900002,1330.0,0,0.0,1.0,6.0,2.0,0.0,1.0,1.0,3.0,6.0,1.571429,2.070197,2.600000,3.091702,1.900000,2.461225,0.0,0.0,0.171429,0.282313
1581,2017-02-02,13669,15,1004550,0.0,1,0,22,1318,0,9,7,2,14,53.549999,1418.0,1,0.0,0.0,1.0,6.0,2.0,0.0,1.0,2.0,0.0,1.428571,2.149197,2.566667,3.114851,1.883333,2.470790,0.0,0.0,0.056122,0.154762
1582,2017-02-03,13669,15,1004550,3.0,0,0,22,1318,0,9,7,2,14,53.810001,1287.0,0,5.0,0.0,0.0,1.0,6.0,2.0,0.0,1.0,0.0,2.000000,2.516611,2.733333,3.106537,1.933333,2.503331,0.0,0.0,0.071429,0.254762
1583,2017-02-04,13669,15,1004550,5.0,0,0,22,1318,0,9,7,2,14,NaN,1372.0,0,1.0,5.0,0.0,0.0,1.0,6.0,2.0,0.0,0.0,2.142857,2.410295,2.766667,3.081461,1.933333,2.503331,0.0,0.0,0.100000,0.159524


After the data is loaded the train data is sorted.

In [ ]:
df = df.sort_values(by=['entity','date'])
df = df.reset_index(drop=True)

In [ ]:
df.drop(['unit_saleslag16', 'unit_saleslag17', 'unit_saleslag18',
       'unit_saleslag19', 'unit_saleslag20', 'unit_saleslag21',
       'unit_saleslag22', 'unit_saleslag30', 'unit_saleslag60', 'mean_7',
       'std_7', 'mean_30', 'std_30', 'mean_60', 'std_60'], axis=1, inplace=True)

cat_vars = ['store_nbr', 'item_nbr', 'onpromotion', 'entity', 'Holiday', 'family', 'class', 'perishable',
       'city', 'state', 'type', 'cluster', 'dcoilwtico_na', 'transactions_na', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
cont_vars = ['dcoilwtico', 'transactions', 'ramp_upHoliday',
       'ramp_downHoliday', 'ramp_uponpromotion', 'ramp_downonpromotion']

df[cont_vars] = df[cont_vars].astype('float64')
df.unit_sales = df.unit_sales.astype('float64')
df['unit_sales'] = df['unit_sales'].astype('float64')

number_imputer = fe.ArbitraryNumberImputer(arbitrary_number=-1)
df = number_imputer.fit_transform(df)

## 3 Model Application

In [ ]:
TARGET = 'unit_sales'
forecast_horizon = 16
group_ids = 'entity'

## 3.9 MQCNN

### 3.9.1 Prepare Splits for NN Model

In [ ]:
df

,date,entity,store_nbr,item_nbr,unit_sales,onpromotion,Holiday,family,class,perishable,city,state,type,cluster,dcoilwtico,transactions,promo_missing,unit_saleslag16,unit_saleslag17,unit_saleslag18,unit_saleslag19,unit_saleslag20,unit_saleslag21,unit_saleslag22,unit_saleslag30,unit_saleslag60,mean_7,std_7,mean_30,std_30,mean_60,std_60,ramp_upHoliday,ramp_downHoliday,ramp_uponpromotion,ramp_downonpromotion,dcoilwtico_na,transactions_na,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2016-07-15,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,45.930000,1275.0,1,2.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,9.0,1.000000,1.527525,1.566667,1.924136,1.966667,2.497230,0.0,0.0,0.370408,0.370408,0,0,2016,7,28,15,4,197,False,False,False,False,False,False,1.468541e+09
1,2016-07-16,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,-1.000000,1219.0,1,1.0,2.0,0.0,0.0,4.0,0.0,0.0,2.0,1.0,1.000000,1.527525,1.533333,1.925032,1.950000,2.500339,0.0,0.0,0.370408,0.370408,1,0,2016,7,28,16,5,198,False,False,False,False,False,False,1.468627e+09
2,2016-07-17,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,-1.000000,885.0,1,1.0,1.0,2.0,0.0,0.0,4.0,0.0,3.0,1.0,1.142857,1.463850,1.533333,1.925032,1.950000,2.500339,0.0,0.0,0.370408,0.370408,1,0,2016,7,28,17,6,199,False,False,False,False,False,False,1.468714e+09
3,2016-07-18,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,45.230000,1445.0,1,2.0,1.0,1.0,2.0,0.0,0.0,4.0,5.0,3.0,1.428571,1.397276,1.433333,1.813424,1.866667,2.410898,0.0,0.0,0.370408,0.370408,0,0,2016,7,29,18,0,200,False,False,False,False,False,False,1.468800e+09
4,2016-07-19,13667,15,1001305,0.0,1,0,12,1016,0,9,7,2,14,44.639999,1360.0,1,0.0,2.0,1.0,1.0,2.0,0.0,0.0,4.0,1.0,0.857143,0.899735,1.433333,1.813424,1.850000,2.420429,0.0,0.0,0.370408,0.370408,0,0,2016,7,29,19,1,201,False,False,False,False,False,False,1.468886e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944544,2017-08-11,174677,9,996122,0.0,1,0,3,1124,0,18,12,1,5,-1.000000,-1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000057,0.216667,0.845560,0.0,0.0,0.297619,0.370408,1,1,2017,8,32,11,4,223,False,False,False,False,False,False,1.502410e+09
2944545,2017-08-12,174677,9,996122,1.0,1,0,3,1124,0,18,12,1,5,-1.000000,-1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000057,0.216667,0.845560,0.0,0.0,0.261905,0.370408,1,1,2017,8,32,12,5,224,False,False,False,False,False,False,1.502496e+09
2944546,2017-08-13,174677,9,996122,0.0,1,0,3,1124,0,18,12,1,5,-1.000000,-1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000057,0.200000,0.839693,0.0,0.0,0.214286,0.370408,1,1,2017,8,32,13,6,225,False,False,False,False,False,False,1.502582e+09
2944547,2017-08-14,174677,9,996122,0.0,1,0,3,1124,0,18,12,1,5,-1.000000,-1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000057,0.183333,0.833446,0.0,0.0,0.142857,0.370408,1,1,2017,8,33,14,0,226,False,False,False,False,False,False,1.502669e+09


For this Neural Network Model the target it not only inferred by the features in the same row, but also by features of "previous" rows as they will be encoded to better predict the upcoming values.
We do not only need to provide the corresponding rows for the target in our forecast horizon , but also the previous features and target values in lookback length in order to predict. This means that we do not split our dataframe as for tabular data.
We create a `full_train_df`, that contains information from start until holdout DATE (train_val split will be done internally in fit method) and a `full_test_df` which contains information from start until end of holdout DATE.

In [ ]:
# inputs required for neural network
full_train_df = df[df['date'] < pd.to_datetime("2017-07-31", format='%Y-%m-%d')].sort_values(['entity', 'date'])
full_test_df = df.sort_values(['entity', 'date'])
display(full_train_df.tail())

date,entity,store_nbr,item_nbr,unit_sales,onpromotion,Holiday,family,class,perishable,city,state,type,cluster,dcoilwtico,transactions,promo_missing,ramp_upHoliday,ramp_downHoliday,ramp_uponpromotion,ramp_downonpromotion,dcoilwtico_na,transactions_na,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
2017-07-26T00:00:00.000+0000,174677,9,996122,0.0,1,0,3,1124,0,18,12,1,5,48.58000183105469,1720.0,1,0.0,0.0,0.37040816326530607,0.37040816326530607,0,0,2017,7,30,26,2,207,false,false,false,false,false,false,1.5010272E9
2017-07-27T00:00:00.000+0000,174677,9,996122,0.0,1,0,3,1124,0,18,12,1,5,49.04999923706055,1717.0,1,0.0,0.0,0.37040816326530607,0.37040816326530607,0,0,2017,7,30,27,3,208,false,false,false,false,false,false,1.5011136E9
2017-07-28T00:00:00.000+0000,174677,9,996122,0.0,1,0,3,1124,0,18,12,1,5,49.720001220703125,1790.0,1,0.0,0.0,0.37040816326530607,0.37040816326530607,0,0,2017,7,30,28,4,209,false,false,false,false,false,false,1.5012E9
2017-07-29T00:00:00.000+0000,174677,9,996122,0.0,1,0,3,1124,0,18,12,1,5,-1.0,2113.0,1,0.0,0.0,0.37040816326530607,0.37040816326530607,1,0,2017,7,30,29,5,210,false,false,false,false,false,false,1.5012864E9
2017-07-30T00:00:00.000+0000,174677,9,996122,0.0,1,0,3,1124,0,18,12,1,5,-1.0,1884.0,1,0.0,0.0,0.37040816326530607,0.37040816326530607,1,0,2017,7,30,30,6,211,false,false,false,false,false,false,1.5013728E9


### 3.9.2 Start Training NN Model

In [ ]:
# DYNAMIC CONTINUOUS
# collect all dynamic real features and convert them to int or float
dynamic_bool_var = ['Holiday', 'dcoilwtico_na', 'transactions_na', 'onpromotion']
full_train_df[dynamic_bool_var] = full_train_df[dynamic_bool_var].astype(int)
full_test_df[dynamic_bool_var] = full_test_df[dynamic_bool_var].astype(int)

dynamic_cont_var = ['dcoilwtico', 'transactions', 'ramp_upHoliday',
       'ramp_downHoliday', 'ramp_uponpromotion', 'ramp_downonpromotion'] #same as for TFT
full_train_df[dynamic_cont_var] = full_train_df[dynamic_cont_var].astype(float)
full_test_df[dynamic_cont_var] = full_test_df[dynamic_cont_var].astype(float)

dynamic_cat_var = ['Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']

# full_train_df[dynamic_cat_var] = full_train_df[dynamic_cat_var].astype(str)
# full_test_df[dynamic_cat_var] = full_test_df[dynamic_cat_var].astype(str)
full_train_df[dynamic_cat_var] = full_train_df[dynamic_cat_var].astype(int)
full_test_df[dynamic_cat_var] = full_test_df[dynamic_cat_var].astype(int)

feat_dynamic_real = dynamic_bool_var+dynamic_cat_var+dynamic_cont_var


# STATIC CATEGORICAL
# bring str_cat_vars to numerical format first
feat_static_cat = ['store_nbr', 'item_nbr', 'entity', 'family', 'class', 'perishable',
       'city', 'state', 'type', 'cluster']
oe = OrdinalEncoder()
# These are all known features therefore we can fit transform test set
full_test_df[feat_static_cat] = oe.fit_transform(full_test_df[feat_static_cat])
full_train_df[feat_static_cat] = oe.transform(full_train_df[feat_static_cat])
full_test_df[feat_static_cat] = full_test_df[feat_static_cat].astype(str)
full_test_df[feat_static_cat] = full_test_df[feat_static_cat].astype(str)

In [ ]:
# obtain_y_test_out_of_X_test()
mqcnn_y_test = MQCNN.obtain_y_test_out_of_X_test(X_test=full_test_df, forecast_horizon=forecast_horizon, timestamp="date", target=TARGET, item_id=group_ids)

In [ ]:
from gluonts.mx.trainer.learning_rate_scheduler import LearningRateReduction
from gluonts.mx.trainer.model_averaging import ModelAveraging, SelectNBestMean, save_epoch_info
from gluonts.mx.trainer import Trainer as MXTrainer


modelaveraging = ModelAveraging(avg_strategy=SelectNBestMean(num_models=1))
# if val metric is not improving for patience epochs then learning rate will be lowered by decay factor 
# --> if this is then below min_lr train will stop immediately
scheduler = LearningRateReduction(patience=5, 
                                  base_lr=0.0001, 
                                  objective='min', 
                                  decay_factor= 0.1, 
                                  min_lr =0.00009) 

trainer = MXTrainer(add_default_callbacks=True, 
                    callbacks=[scheduler, modelaveraging], 
                    clip_gradient=10.0, 
                    ctx="gpu", 
                    epochs=100, 
                    hybridize=False, # for gpu 
                    num_batches_per_epoch=100, 
                    weight_decay = 1e-08)

mqcnn_params = {'batch_size': 256, 
                'num_forking' : None, 
                'decoder_mlp_dim_seq' : [32, 32], 
                'channels_seq' : [32,32,32,32,32], 
                'dilation_seq' : [1,2,4,8,16],
                'kernel_size_seq' : [2,2,2,2,2], 
                'scaling_decoder_dynamic_feature' : False,
                'scaling': False 
                }

mqcnn_params['trainer'] = trainer

start_time = time.perf_counter()

mqcnn_reg = MQCNN(freq = "D", lookback=lookback, forecast_horizon=forecast_horizon, item_id=group_ids, 
            timestamp="date", feat_static_cat=feat_static_cat, feat_dynamic_real=cont_vars,
            past_feat_dynamic_real=None, cardinality_static_cat=[len(full_train_df[var].unique()) for var in feat_static_cat], 
            dynamic_feature_scaler=StandardScaler(), quantiles=[0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95])
mqcnn_reg.fit(full_train_df, TARGET, params_mqcnn=mqcnn_params, verbose = True)
mqcnn_pred = mqcnn_reg.predict(full_test_df, prediction_types=[PredEnum.POINT_ESTIMATES, PredEnum.QUANTILES])
mqcnn_metrics = mqcnn_reg.metrics(mqcnn_y_test, mqcnn_pred, confidence_interval_quantiles=[0.1,0.9])
mqcnn_metrics['rmsle'] = np.sqrt((( np.log1p(np.reshape(mqcnn_y_test, (-1,))) - np.log1p(np.reshape(mqcnn_pred[PredEnum.POINT_ESTIMATES], newshape=(-1,))) )**2).mean())

end_time = time.perf_counter()
full_time = np.round(end_time - start_time, 2)
mqcnn_metrics['time'] = full_time

e6900b37e7e746c4ae29409c5238c3f0
100%|██████████| 100/100 [00:36<00:00,  2.70it/s, epoch=1/100, avg_epoch_loss=0.0407]
29it [00:01, 20.94it/s, epoch=1/100, validation_avg_epoch_loss=0.03]
100%|██████████| 100/100 [00:36<00:00,  2.72it/s, epoch=2/100, avg_epoch_loss=0.0279]
29it [00:01, 20.90it/s, epoch=2/100, validation_avg_epoch_loss=0.0262]
100%|██████████| 100/100 [00:37<00:00,  2.70it/s, epoch=3/100, avg_epoch_loss=0.0254]
29it [00:01, 20.97it/s, epoch=3/100, validation_avg_epoch_loss=0.0249]
100%|██████████| 100/100 [00:36<00:00,  2.71it/s, epoch=4/100, avg_epoch_loss=0.0244]
29it [00:01, 21.02it/s, epoch=4/100, validation_avg_epoch_loss=0.0236]
100%|██████████| 100/100 [00:37<00:00,  2.70it/s, epoch=5/100, avg_epoch_loss=0.0228]
29it [00:01, 21.04it/s, epoch=5/100, validation_avg_epoch_loss=0.0225]
100%|██████████| 100/100 [00:36<00:00,  2.72it/s, epoch=6/100, avg_epoch_loss=0.0221]
29it [00:01, 21.03it/s, epoch=6/100, validation_avg_epoch_loss=0.0214]
100%|██████████| 100/100 [0

In [ ]:
mqcnn_metrics

Out[30]: {'mse': 330.7649087957658,
 'mae': 2.932252782473539,
 'rmse': 18.186943360437613,
 'mape': 825964192231518.2,
 'rmspe': 7460122157535074.0,
 'avg_interval_length': 7.4368057,
 'coverage': 0.24607459838704138,
 'time': 2838.55}